In [ ]:
!pip install --upgrade huggingface_hub datasets pillow albumentations 

In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

import tensorflow as tf
import albumentations

from huggingface_hub import login
login("***")

import numpy as np

from datasets import load_dataset

from tensorflow.keras.layers import Conv2D, Flatten, Dense, Dropout, BatchNormalization, MaxPooling2D, Softmax
from tensorflow.keras import Model

import warnings

# Suppress all FutureWarnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# Suppress all UserWarnings
warnings.simplefilter(action='ignore', category=UserWarning)

In [ ]:
tf.config.run_functions_eagerly(True)
print("TensorFlow version:", tf.__version__)
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

In [ ]:
train_dataset = load_dataset("marmal88/skin_cancer", split="train")
valid_dataset = load_dataset("marmal88/skin_cancer", split="validation")
test_dataset  = load_dataset("marmal88/skin_cancer", split="test")

In [ ]:
dx_to_num = {
    "melanocytic_Nevi":0,
    "melanoma":1,
    "benign_keratosis-like_lesions":2,
    "basal_cell_carcinoma":3,
    "actinic_keratoses":4,
    "vascular_lesions":5,
    "dermatofibroma":6
}

In [ ]:
transform = albumentations.Compose([albumentations.Resize(width=128, height=128)])
def transforms(examples):
    examples["pixel_values"] = [(transform(image=np.array(image))["image"] / 255.0) for image in examples["image"]]
    examples["labels"] = [dx_to_num[dx] for dx in examples["dx"]]
    return examples

train_dataset.set_transform(transforms)
valid_dataset.set_transform(transforms)
test_dataset.set_transform(transforms)

In [ ]:
train_ds = train_dataset.to_tf_dataset(columns=["pixel_values"], label_cols=["labels"], shuffle=True, batch_size=32)
valid_ds = valid_dataset.to_tf_dataset(columns=["pixel_values"], label_cols=["labels"], shuffle=True, batch_size=32)
test_ds = test_dataset.to_tf_dataset(columns=["pixel_values"], label_cols=["labels"], shuffle=True, batch_size=32)

In [ ]:
class MyModel(Model):
    def __init__(self):
        super().__init__()
        self.conv1 = Conv2D(filters=32, kernel_size=3, activation='relu')
        self.bn1 = BatchNormalization()
        self.pool1 = MaxPooling2D(pool_size=(2, 2))
        
        self.conv2 = Conv2D(filters=64, kernel_size=3, activation='relu')
        self.bn2 = BatchNormalization()
        self.pool2 = MaxPooling2D(pool_size=(2, 2))
        
        self.flatten = Flatten()
        self.d1 = Dense(256, activation='relu', kernel_regularizer='l2')
        self.bn3 = BatchNormalization()
        self.d2 = Dense(64, activation='relu', kernel_regularizer='l2')
        self.bn4 = BatchNormalization()
        self.d3 = Dropout(0.5)
        self.d4 = Dense(7, activation='softmax')

    def call(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.pool1(x)
        
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.pool2(x)
        
        x = self.flatten(x)
        x = self.d1(x)
        x = self.bn3(x)
        x = self.d2(x)
        x = self.bn4(x)
        x = self.d3(x)
        return self.d4(x)

model = MyModel()

In [ ]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False)

optimizer = tf.keras.optimizers.Adam()

train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

valid_loss = tf.keras.metrics.Mean(name='valid_loss')
valid_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='valid_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

In [ ]:
@tf.function
def train_step(images, labels):
    with tf.GradientTape() as tape:
        predictions = model(images, training=True)
        loss = loss_object(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    train_loss(loss)
    train_accuracy(labels, predictions)

@tf.function
def valid_step(images, labels):
    with tf.GradientTape() as tape:
        predictions = model(images, training=True)
        v_loss = loss_object(labels, predictions)
    gradients = tape.gradient(v_loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    valid_loss(v_loss)
    valid_accuracy(labels, predictions)

@tf.function
def test_step(images, labels):
    predictions = model(images, training=False)
    t_loss = loss_object(labels, predictions)

    test_loss(t_loss)
    test_accuracy(labels, predictions)

In [ ]:
EPOCHS = 10

for epoch in range(EPOCHS):
    train_loss.reset_state()
    train_accuracy.reset_state()
    valid_loss.reset_state()
    valid_accuracy.reset_state()
    test_loss.reset_state()
    test_accuracy.reset_state()

    for images, labels in train_ds:
        train_step(images, labels)

    for valid_images, valid_labels in valid_ds:
        valid_step(valid_images, valid_labels)

    for test_images, test_labels in test_ds:
        test_step(test_images, test_labels)

    print(f'Epoch {epoch + 1}, '
          f'Loss: {train_loss.result():.2f}, '
          f'Accuracy: {train_accuracy.result() * 100:.2f}%, '
          f'Validation Loss: {valid_loss.result():.2f}, '
          f'Validation Accuracy: {valid_accuracy.result() * 100:.2f}%, '
          f'Test Loss: {test_loss.result():.2f}, '
          f'Test Accuracy: {test_accuracy.result() * 100:.2f}%')

Best: 99.14